In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
loans_df = pd.read_csv("Resources/lending_data.csv")

# Review the DataFrame
loans_df.head(2)

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0


In [3]:
#obtain data about the dat set
loans_df.shape

(77536, 8)

### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [4]:
# Separate the data into labels and features
# Separate the y variable, the labels
y = loans_df['loan_status']

# Separate the X variable, the features
X = loans_df.copy()
X.drop(columns="loan_status", inplace=True)

In [5]:
# Review the y variable Series
y[:5]

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [6]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Split the data into training and testing datasets by using `train_test_split`.

In [7]:
# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [8]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using training data
classifier.fit(X_train, y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [9]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Generate a confusion matrix.

* Print the classification report.

In [10]:
# Generate a confusion matrix for the model
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["LR Predicted 0", "LR Predicted 1"]
)

In [11]:
# Print the classification report for the model
print("Confusion Matrix")
display(cm_df)
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,LR Predicted 0,LR Predicted 1
Actual 0,18663,102
Actual 1,56,563


Classification Report
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.85      0.91      0.88       619

    accuracy                           0.99     19384
   macro avg       0.92      0.95      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic regression model with this data has a sensitivity score of 99.7% and a specificity 84.66% for healthy loans, meaning that 99.7% of the healthy loans were correctly identified as healthy and 84% of the loans that were in danger of defaulting were correctly identified as being at risk. If the bank is concerned with not turning away people who will pay off the loan, this model does a great job, if the bank is more concerned with not lending to people who might go into default, this model is a little less accurate than the bank might be comfortable with. Please continue reading to see if using a KMeans method or Random Forests might be a better option.



In [12]:
#import dependencies to run KMEans and Random Forests to identify of one of these is a better predictor for this data
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [14]:
# Initialize the K-Means model with n_clusters=2, 2 clusters are chosen as the label catagorey is binary
k_model = KMeans(n_clusters=2)

#Initilaize the random forest model 
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier()

In [15]:
#fit the model, first for KMeans, then Random Forest
k_model.fit(X_train, y_train)


C:\Users\krmce\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=2)

In [16]:
rf_model.fit(X_train, y_train)

RandomForestClassifier()

In [17]:
#make predictions, first for KMeans, then Random Forest
k_predictions = k_model.predict(X_test)
rf_predictions = rf_model.predict(X_test)

In [18]:
# Generate a confusion matrix for the KMeans model
k_cm = confusion_matrix(y_test, k_predictions)
k_cm_df = pd.DataFrame(
    k_cm, index=["Actual 0", "Actual 1"], columns=["KMeans Predicted 0", "KMeans Predicted 1"]
)

# Generate a confusion matrix for the Random Forest model

rf_cm = confusion_matrix(y_test, rf_predictions)
rf_cm_df = pd.DataFrame(
    rf_cm, index=["Actual 0", "Actual 1"], columns=["RF Predicted 0", "RF Predicted 1"]
)

#merge the three confusion matrices together 
merged_cm = pd.concat([cm_df, k_cm_df, rf_cm_df], axis=1)

#print the merged confusion matrix and the classification reports for hte three different models
print("Confusion Matrix")
display(merged_cm)
print("Classification Report for Logistic Regression")
print(classification_report(y_test, predictions))
print("Classification Report for KMeans")
print(classification_report(y_test, k_predictions))
print("Classification Report for Random Forest")
print(classification_report(y_test, rf_predictions))

Confusion Matrix


,LR Predicted 0,LR Predicted 1,KMeans Predicted 0,KMeans Predicted 1,RF Predicted 0,RF Predicted 1
Actual 0,18663,102,18650,115,18665,100
Actual 1,56,563,4,615,64,555


Classification Report for Logistic Regression
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.85      0.91      0.88       619

    accuracy                           0.99     19384
   macro avg       0.92      0.95      0.94     19384
weighted avg       0.99      0.99      0.99     19384

Classification Report for KMeans
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.84      0.99      0.91       619

    accuracy                           0.99     19384
   macro avg       0.92      0.99      0.95     19384
weighted avg       0.99      0.99      0.99     19384

Classification Report for Random Forest
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.85      0.90      0.87       619

    accuracy                           0.99     19384
   macro 

***After all running the data through a KMeans model and a Random Forest model, I was able to assess that the Logistic Regression is the better model for making predictions with the given data set. The Random Forest model came close but it incorrectly identified more people who were in danger of defaulting as being healthy loans.